### Script to convert LANDIS Biomass to forest LAI

Outputs a geoTIFF for ingestion into GRASS and RHESSys

Blank cells (biomass of 0) are assumed to be grass and given an LAI of 1, this is an assumption that will need to be documented in the methods.

In [1]:
def biomass2LAI(x,sla,leaf_stem):
    '''
    Convert biomass to lai based on the specific leaf area.
    
    x: array of biomass values in g/m2
    sla: specific leaf area m2/kg c
    leaf_stem: ratio of leaf mass to stem mass
    '''
    x /= 1000. # g/m2 > kg/m2
    leaf = (x*leaf_stem) / (1.+leaf_stem) # compute leaf wet biomass
    lai = leaf * sla # convert leaf dry biomass to leaf area

    return lai

In [2]:
import gdal
import glob

In [3]:
rastpth = './data/LAI_Boulder_crop.tif'
ds = gdal.Open(rastpth)
rast = np.array(ds.GetRasterBand(1).ReadAsArray(),dtype=float)
rast[rast<0] = np.NaN
mask = rast.copy()
mask[np.isnan(mask)==0] = 1

In [4]:
files = glob.glob('./data/LANDIS_output/climate_change/*/*/*.tiff')

In [5]:
dat = pd.DataFrame({'filename':files})

In [6]:
def get_run(fl):
    return int(fl.split("/")[-2].split('_')[-1])

In [7]:
def get_scenario(fl):
    
    key = fl.split("/")[-2][:-2]
    
    if key == 'BDA+FIRE_+4C_minus15%precip':
        out = 'PGW_LP'
    
    if key == 'BDA+FIRE_+4C_+15%precip':
        out = 'PGW_MP'
        
    return out 

In [8]:
def get_year(fl):
    
    yr = int(fl.split('/')[-1].split('-')[-1].split('_')[0])
    yr -= 10
    yr += 2000
    
    return yr

In [9]:
dat['run'] = dat.filename.map(get_run)
dat['scenario'] = dat.filename.map(get_scenario)
dat['year'] = dat.filename.map(get_year)

In [10]:
dat = dat.loc[(dat.year==2000) | (dat.year==2050) | (dat.year==2100)]
dat

,filename,run,scenario,year
1,./data/LANDIS_output/climate_change/PGW_less_P...,1,PGW_LP,2000
3,./data/LANDIS_output/climate_change/PGW_less_P...,1,PGW_LP,2100
8,./data/LANDIS_output/climate_change/PGW_less_P...,1,PGW_LP,2050
13,./data/LANDIS_output/climate_change/PGW_less_P...,2,PGW_LP,2000
15,./data/LANDIS_output/climate_change/PGW_less_P...,2,PGW_LP,2100
20,./data/LANDIS_output/climate_change/PGW_less_P...,2,PGW_LP,2050
25,./data/LANDIS_output/climate_change/PGW_less_P...,5,PGW_LP,2000
27,./data/LANDIS_output/climate_change/PGW_less_P...,5,PGW_LP,2100
32,./data/LANDIS_output/climate_change/PGW_less_P...,5,PGW_LP,2050
37,./data/LANDIS_output/climate_change/PGW_more_P...,1,PGW_MP,2000


In [11]:
len(dat)

18

In [12]:
sla = 9. # specific leaf area
leaf_stem = 0.0141 # ratio of leaf to stem biomass

In [13]:
def biomass2LAItiff(fl,sla,leaf_stem,outpath1,outpath2,outpath3,outtiff):
    ds = gdal.Open(fl)
    rast = np.array(ds.GetRasterBand(1).ReadAsArray(),dtype=float)

    srs = ds.GetProjection() # get the projection of the input data set
    transform = ds.GetGeoTransform() # get the limits and resolution of the input data set
    
    # generate lai map
    lai = biomass2LAI(rast,sla,leaf_stem)
    lai[lai==0] = 0.5 # if there is a blank cell, make it into grass with an LAI of 0.5 Blanken et al 2009
    
    # generate land cover type map
    lc = rast.copy()
    lc[lc==0] = 2 # tundra (grass)
    lc[lc!=2] = 4 # forest (evergreen)
    
    # generate land use type map
    lu = lc.copy()
    lu[lu==2] = 11 # alpine
    lu[lu==4] = 12 # subalpine
    
    m,n = np.shape(lai) # get the dimensions of the DEM
    
    # output lai maps
    if outtiff:
        output = gdal.GetDriverByName('GTiff').Create(outpath1,n,m,1,gdal.GDT_Float32) # open the output file
        output.SetGeoTransform(transform) # set coordinates
        output.SetProjection(srs) # set projection
    
        output.GetRasterBand(1).WriteArray(lai)
        output = None # close dataset
    
    # output land cover type map
    if outtiff:
        output = gdal.GetDriverByName('GTiff').Create(outpath2,n,m,1,gdal.GDT_Int16) # open the output file
        output.SetGeoTransform(transform) # set coordinates
        output.SetProjection(srs) # set projection
    
        output.GetRasterBand(1).WriteArray(lc)
        output = None # close dataset
    
    # output land use map
    if outtiff:
        output = gdal.GetDriverByName('GTiff').Create(outpath3,n,m,1,gdal.GDT_Int16) # open the output file
        output.SetGeoTransform(transform) # set coordinates
        output.SetProjection(srs) # set projection
    
        output.GetRasterBand(1).WriteArray(lu)
        output = None # close dataset
    

In [14]:
def run_laitiff(dat):
    fl = dat.filename
    
    outpath1 = './data/RHESSys_scenarios/LANDIS_LAI_%s_%s_%s.tiff'%(dat.scenario,dat.run,dat.year)
    outpath2 = './data/RHESSys_scenarios/LANDIS_landcover_%s_%s_%s.tiff'%(dat.scenario,dat.run,dat.year)
    outpath3 = './data/RHESSys_scenarios/LANDIS_landuse_%s_%s_%s.tiff'%(dat.scenario,dat.run,dat.year)
    
    biomass2LAItiff(fl,sla,leaf_stem,outpath1,outpath2,outpath3,True)
    
    return None

In [15]:
dat.apply(run_laitiff,axis=1)

1     None
3     None
8     None
13    None
15    None
20    None
25    None
27    None
32    None
37    None
39    None
44    None
49    None
51    None
56    None
61    None
63    None
68    None
dtype: object